# **Forecasting Commercial Banks Transactions**

<ol><i>Authors</i>:
    <li>Nobert Akwir</li>
    <li>Karen Amanya</li>
    <li>Eugene Kuloba</li>
    <li>David Mwiti</li>
    <li>Anthony Nene</li>
</ol>

## **1. Business Understanding**

<img src = 'Images/bank-img.jpg' ></img>

The Banking sector in Kenya has been seeing immense growth over the past few years, this being in direct relation to the penetration rate/ improved access to banking services by major banks we have in Kenya. From the Central Bank’s Financial access report(2021), 29.6% of the total population of Kenyans having formal access to bank services still used physical bank branches.

As this penetration rate has increased, the level of customer interaction and transactions at the branch level has also increased, hence leading to branches requesting more cash from their central vaults, and also repatriating more cash back to the central vault. This sometimes is not the norm, as the branches have to breach their vault limit due to more transactions by the customers even after repatriation is done(which are always in the morning hours), or even having requested less cash hence them not having enough cash to serve the customers.

The branches holding this excess cash leads the bank to incur more expenses as they have to take a cover for the excess cash, and also this is large risk exposure to the bank if the branches get hit. For the less cash perspective, this brings dissatisfaction to their customers and brings down the rating of the bank.

NCBA Bank is a commercial banking company that aims to conduct research to get findings that aid in confirming how much their branches should hold at specific periods in order for them to have sufficient funds to carry out its business with its customers and also not breach its vaults limits by too much.This project aims to develop a time series analysis model that will describe and forecast future transaction thresholds, to assist the bank’s branches identify the amount of cash they need at specific periods of time in order to prevent being over limit and also have satisfactory cash to serve their customers.


### **Objectives**

> #### **General Objective**
The general objective is to develop a model that will successfully forecast cash transactions that will help NCBA Bank maintain a sufficient cash at its specifc branches.
> #### **Specific Objectives**
<ul type='square'>
    <li >  </li>
    <li> </li>
    <li></li>
</ul>


### **Success Criteria**

## **2. Data Understanding**

The data used in this project is sources from data world. The data consists of demo bank transaction details(debits and credits) carried out by different customers and includes other details such as the amount, the mode of transaction(either cash, RTGS, EFT, SWIFT e.t.c), the account's balance at the time of the transation, the time of the transaction e.t.c

#### **Importing the required libraries**

In [1]:
import pandas as pd

In [3]:
raw_df = pd.read_csv('Data/completedtrans.csv')
raw_df.head()

C:\Users\Karen\AppData\Local\Temp\ipykernel_3268\3692760457.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('Data/completedtrans.csv')


,Unnamed: 0,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,year,month,day,fulldate,fulltime,fulldatewithtime
0,1,T00695247,A00002378,Credit,Credit in Cash,700.0,700.0,NaN,NaN,NaN,2013,1,1,2013-01-01,11:02:40,2013-01-01T11:02:40
1,2,T00171812,A00000576,Credit,Credit in Cash,900.0,900.0,NaN,NaN,NaN,2013,1,1,2013-01-01,08:23:33,2013-01-01T08:23:33
2,3,T00207264,A00000704,Credit,Credit in Cash,1000.0,1000.0,NaN,NaN,NaN,2013,1,1,2013-01-01,15:19:21,2013-01-01T15:19:21
3,4,T01117247,A00003818,Credit,Credit in Cash,600.0,600.0,NaN,NaN,NaN,2013,1,1,2013-01-01,10:42:35,2013-01-01T10:42:35
4,5,T00579373,A00001972,Credit,Credit in Cash,400.0,400.0,NaN,NaN,NaN,2013,1,2,2013-01-02,11:39:25,2013-01-02T11:39:25


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 16 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Unnamed: 0        1056320 non-null  int64  
 1   trans_id          1056320 non-null  object 
 2   account_id        1056320 non-null  object 
 3   type              1056320 non-null  object 
 4   operation         873206 non-null   object 
 5   amount            1056320 non-null  float64
 6   balance           1056320 non-null  float64
 7   k_symbol          574439 non-null   object 
 8   bank              273508 non-null   object 
 9   account           295389 non-null   float64
 10  year              1056320 non-null  int64  
 11  month             1056320 non-null  int64  
 12  day               1056320 non-null  int64  
 13  fulldate          1056320 non-null  object 
 14  fulltime          1056320 non-null  object 
 15  fulldatewithtime  1056320 non-null  object 
dtype

From our objectives, the features we will need are the amount, operatio(which describe the nature of the transaction) and fulldate and fulltime features.
One thing we have to note, is that we will be only looking into transaction operations that deal with physical cash.

In [5]:
#Checking unique values in the operation's feature.
raw_df['operation'].unique()

array(['Credit in Cash', 'Collection from Another Bank',
       'Cash Withdrawal', nan, 'Remittance to Another Bank',
       'Credit Card Withdrawal'], dtype=object)

As per above array of unique values in our operation's feature of the dataset, the categories that deal with physical cash are the "Credit in Cash" and "Cash Withdrawal". The "Collection from Another Bank" and "Remittance to another Bank" rarely deal with hard cash as they are mostly transfered through Real Time Gross Settlement. As for the "Credit Card Withdrawal", most of this transactions are done through the ATMs, and the ATMs cash are handled at a different vault and not the branches vault therefore it not be ideal to consider them fro our model.

In [6]:
#Checking the value counts of the unique categories in the "operation"column
raw_df['operation'].value_counts()

Cash Withdrawal                 434918
Remittance to Another Bank      208283
Credit in Cash                  156743
Collection from Another Bank     65226
Credit Card Withdrawal            8036
Name: operation, dtype: int64

In [7]:
#splitting the original dataframe with regards to the operation column to get the credits dataset
credits_df = raw_df[raw_df['operation']=='Credit in Cash']
credits_df = credits_df[['amount','fulldate','fulltime']]
credits_df.shape

(156743, 3)

In [8]:
#Noticed there were values indicating 60 mins and 60 seconds in the fulltime column.
#We have to correct this to allow us change concated column(fulldate & fulltime), into datetime type.
credits_df['fulltime'] = credits_df['fulltime'].str.replace(":60:",":59:")
credits_df['fulltime'] = credits_df['fulltime'].str.replace(":60",":59")

In [9]:
#Concating the fulldate and fulltime columns then channging the datatype to datetime
#droping the fulldate and fulltime columns
credits_df['date_time'] = pd.to_datetime(credits_df['fulldate'] +" " +credits_df['fulltime'])
credits_df.drop(['fulldate','fulltime'],axis=1,inplace=True)

In [10]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156743 entries, 0 to 1051813
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   amount     156743 non-null  float64       
 1   date_time  156743 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.6 MB
